In [ ]:
import numpy as np
import gym
from DQN_Utils import make_env, DQNAgent

In [ ]:
env = gym.make("CarRacing-v2", continuous=False)
env = make_env(env)
best_reward = -np.inf
agent = DQNAgent(gamma=0.99, epsilon=1.0, learning_rate=2.5e-4,
                 input_shape=(env.observation_space.shape),
                 n_actions=env.action_space.n,
                 memory_size=20000,
                 epsilon_decay=1.5e-6,
                 epsilon_min=0.1)
n_steps = int(2e6)

In [ ]:
reward_per_episode = []
max_episode_steps = 1000
n = 1
episode = 1
while n < n_steps:
    done = False
    episode_reward = 0
    episode_step = 0
    neg_reward_cnt = 0
    s = env.reset()
    while not done and episode_step < max_episode_steps:
        a = agent.choose_action(s)
        s2, r, done, _, _ = env.step(a)
        episode_step += 1
        episode_reward += r
        agent.remember(s, a, r, s2, int(done))
        agent.learn()
        s = s2
        n += 1
        if r < 0 and episode_step>300:
            neg_reward_cnt += 1
        else:
            neg_reward_cnt = 0
        
        if neg_reward_cnt > 30:
            break
            
        if episode_reward < -10:
            break
            
    reward_per_episode.append(episode_reward)
    best_reward = max(best_reward, episode_reward)

    if episode == 1 or episode % 50 == 0:
        avg_reward = np.mean(reward_per_episode[-50:])
        print_string = f'[ Episode: {episode} | Step: {n} / {n_steps} | 50-Episode Average Score: {avg_reward:.2f} | Best Score: {best_reward} | Epsilon: {agent.epsilon}]'
        print(print_string)
        agent.create_checkpoint(path='checkpoints/')
        with open('CarRacing_DQN_training_logs.csv', 'a') as f:
            f.write(f'{episode},{n},{avg_reward},{best_reward},{agent.epsilon}\n')
    
    episode += 1
env.close()